In [1]:
import sys
sys.path.append('../')
from gymenv_v2 import make_multiple_env
import numpy as np


import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical

In [111]:
class Policy(nn.Module):
    def __init__(self, num_inputs):
        super(Policy, self).__init__()
        self.batchNormMatrix = nn.BatchNorm1d(num_features = num_inputs)
        
        self.embedding = nn.Embedding(num_inputs, num_inputs)
        
        self.core = nn.Sequential(
                                nn.Conv1d(1, 16, 3),
                                nn.ELU(),
                                nn.Conv1d(16, 16, 3),
                                nn.ELU(),
                                nn.Conv1d(16, 1, 3),
                                nn.ELU(),
                                )
        
        self.saved_log_probs = []
        self.rewards = []

    def forward(self, s):
        a, b, c, d, e = self._preproc(s)
        a, d = [self.batchNormMatrix(x) for x in [a, d]]
        b, e =[(x - x.min()) / (x.max() - x.min()) for x in [b, e]]
        X, Y = [torch.cat((x, y.unsqueeze(1)), 1) for x, y in zip([a, d], [b, e])]
        
        X, Y = [x.unsqueeze(1) for x in [X, Y]]
        
        H, G = [self.core(x) for x in [X, Y]]
        

        H, G = [x.squeeze(1) for x in [H, G]]
        
        c = self.embedding(torch.LongTensor(c))
        
        print(X.shape, c.shape, Y.shape)
        
        S = H @ G.T
        
        
        action_scores = S.mean(0)
        
        return F.softmax(action_scores, dim=-1)
    
    
    def _preproc(self, s):
        return [torch.FloatTensor(item) for item in s]

In [112]:
easy_config = {
    "load_dir"        : '../instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 10,
    "reward_type"     : 'obj'
}


In [113]:
model = Policy(60)

In [114]:
model(s)

TypeError: expected Long (got Float)

In [54]:
class Observer(object):
    def __init__(self, env_config):
        self.env = make_multiple_env(**env_config)
    def run_episode(self, agent):
        state, ep_reward, d = self.env.reset(), 0, False
        while not d:
            # send the state to the agent to get an action
            action = agent.select_action(state)

            # apply the action to the environment, and get the reward
            state, reward, d, _ = self.env.step(action)
            # report the reward to the agent for training purpose
            agent.report_reward(reward)
            

In [55]:
class Agent(object):
    def __init__(self, num_inputs, training_config,oberver):
        learning_rate = training_config['lr']
        gamma = training_config['gamma']
        
        self.observer = observer
        self.rewards = []
        self.gamma = gamma
        self.policy = Policy(num_inputs)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=learning_rate)
        self.eps = np.finfo(np.float32).eps.item()
        self.save_log_probs = []
        
    def select_action(self, state):
        probs = self.policy(state)
        m = Categorical(probs)
        action = m.sample()
        self.save_log_probs.append(m.log_prob(action))
        return action.item()
    def report_reward(self, reward):
        self.rewards.append(reward)
        
    def run_episode(self):
        self.observer.run_episode(self)
        
    def finish_episode(self):
        R, probs, rewards = 0, self.save_log_probs.copy(), self.rewards.copy()
        
        reward = sum(self.rewards)
        
        self.rewards = []
        self.save_log_probs = []
        
        policy_loss, returns = [], []
        
        for r in rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + self.eps)
        
        for log_prob, R in zip(probs, returns):
            policy_loss.append(-log_prob * R)
        self.optimizer.zero_grad()
        
        policy_loss = torch.stack(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()
        return reward

In [56]:
observer = Observer(easy_config)

loading training instances, dir ../instances/train_10_n60_m60 idx 0
loading training instances, dir ../instances/train_10_n60_m60 idx 1
loading training instances, dir ../instances/train_10_n60_m60 idx 2
loading training instances, dir ../instances/train_10_n60_m60 idx 3
loading training instances, dir ../instances/train_10_n60_m60 idx 4
loading training instances, dir ../instances/train_10_n60_m60 idx 5
loading training instances, dir ../instances/train_10_n60_m60 idx 6
loading training instances, dir ../instances/train_10_n60_m60 idx 7
loading training instances, dir ../instances/train_10_n60_m60 idx 8
loading training instances, dir ../instances/train_10_n60_m60 idx 9


In [57]:
agent = Agent(60, {'lr': 1e-2, 'gamma': .99}, observer)

In [60]:
agent.run_episode()
agent.run_episode()

In [65]:
lst = [1, 2, np.NaN, 3]

In [59]:

for i in lst:
    
    

0.6151751669249279

In [68]:
np.isnan(np.NaN)

True

In [115]:
import json

In [120]:
with open('./tmp.json', 'w') as f:
    json.dump(json.dumps('a'), f)

In [121]:
!ls

__pycache__                       gru_emb_noMC.py
build_multiprocessingplay.ipynb   my_optim.py
debug.ipynb                       playground-Copy1.ipynb
dev.ipynb                         playground.ipynb
devv.ipynb                        rebuildpolicy.ipynb
embedding-Copy1.ipynb             reconstruct.ipynb
embedding-valueapprox-Copy1.ipynb rpc.ipynb
embedding-valueapprox.ipynb       tmp.json
embedding.ipynb                   wandb


In [122]:
import wandb

wandb.login()

run=wandb.init(project="finalproject", entity="ieor-4575", tags=["training-easy"])

wandb: Currently logged in as: ieor-4575 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.26 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [123]:
run.finish()

In [124]:
run=wandb.init(project="finalproject", entity="ieor-4575", tags=["training-easy"])

wandb: wandb version 0.10.26 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [125]:
run.finish()